In [1]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate();

  Activating project at `~/Documents/eth_courses/notebooks/others/julia`


In [2]:
using BenchmarkTools

# Compiler optimizes in simple scenarios 

In [12]:
function type_stable()
    val = 1.0
    for i in 0:1:10
        val /= rand()
    end
    return val
end

@btime type_unstable
@btime type_stable

  1.654 ns (0 allocations: 0 bytes)
  1.655 ns (0 allocations: 0 bytes)


type_stable (generic function with 1 method)

# Global Scope

Non-const variables in global scope are type  unstable

In [12]:
# non-const global
a = 0.0
x,y = rand(1000), rand(1000)

# all global without function
@btime begin
    for i in 1:length(x)
        global a += x[i]^2 + y[i]^2
    end
end

# all global with function
function global_var_1()
    for i in 1:length(x)
        global a += x[i]^2 + y[i]^2
    end

    return a
end

# a local, x and y global 
function global_var_2()
    a = zero(eltype(x))
    for i in 1:length(x)
        a += x[i]^2 + y[i]^2
    end

    return a
end

# all local, all types defined 
function local_var_1(x::Vector{Float64}, y::Vector{Float64})
    a = zero(eltype(x))
    for i in 1:length(x)
        a += x[i]^2 + y[i]^2
    end

    return a
end

# all local, all types defined except variable a 
function local_var_2(x::Vector{Float64}, y::Vector{Float64})
    a::Float64 = zero(eltype(x))
    for i in 1:length(x)
        a += x[i]^2 + y[i]^2
    end

    return a
end

@btime global_var_1()
@btime global_var_2()
@btime local_var_1(x,y)
@btime local_var_2(x,y)

  202.395 μs (7980 allocations: 140.33 KiB)
  192.371 μs (7980 allocations: 140.33 KiB)
  197.231 μs (7980 allocations: 140.33 KiB)
  984.533 ns (0 allocations: 0 bytes)
  984.667 ns (0 allocations: 0 bytes)


659.7108774950834

## Typed global scope

In [15]:
# non-const global
a_::Float64 = 0.0
x_::Vector{Float64},y_::Vector{Float64} = rand(1000), rand(1000)
    
# all global without function
@btime begin
    for i in 1:length(x_)
        global a_ += x_[i]^2 + y_[i]^2
    end
end

# all global with function
function global_var_1()
    for i in 1:length(x_)
        global a_ += x_[i]^2 + y_[i]^2
    end

    return a_
end

# a local, x_ and y_ global 
function global_var_2()
    a = zero(eltype(x_))
    for i in 1:length(x_)
        a += x_[i]^2 + y_[i]^2
    end

    return a
end

# all local, all types defined 
function local_var_1(x_::Vector{Float64}, y_::Vector{Float64})
    a = zero(eltype(x_))
    for i in 1:length(x_)
        a += x_[i]^2 + y_[i]^2
    end

    return a
end

# all local, all types defined except variable a 
function local_var_2(x_::Vector{Float64}, y_::Vector{Float64})
    a::Float64 = zero(eltype(x_))
    for i in 1:length(x_)
        a += x_[i]^2 + y_[i]^2
    end

    return a
end

@btime global_var_1()
@btime global_var_2()
@btime local_var_1(x_,y_)
@btime local_var_2(x_,y_)


  7.590 μs (1000 allocations: 15.62 KiB)
  7.910 μs (1000 allocations: 15.62 KiB)
  982.133 ns (0 allocations: 0 bytes)
  981.250 ns (0 allocations: 0 bytes)
  981.188 ns (0 allocations: 0 bytes)


657.4188025475605

# Tool to check type stability

In [50]:
@code_warntype global_var_1()

MethodInstance for global_var_1()
  from global_var_1() @ Main In[44]:14
Arguments
  #self#::Core.Const(global_var_1)
Locals
  @_2::Any
  i::Any
Body::Any
1 ─ %1  = Main.length(Main.x)::Any
│   %2  = (1:%1)::Any
│         (@_2 = Base.iterate(%2))
│   %4  = (@_2 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_2::Any
│         (i = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Any
│         nothing
│   %11 = Main.a::Any
│   %12 = Main.:^::Core.Const(^)
│   %13 = Base.getindex(Main.x, i)::Any
│   %14 = Core.apply_type(Base.Val, 2)::Core.Const(Val{2})
│   %15 = (%14)()::Core.Const(Val{2}())
│   %16 = Base.literal_pow(%12, %13, %15)::Any
│   %17 = Main.:^::Core.Const(^)
│   %18 = Base.getindex(Main.y, i)::Any
│   %19 = Core.apply_type(Base.Val, 2)::Core.Const(Val{2})
│   %20 = (%19)()::Core.Const(Val{2}())
│   %21 = Base.literal_pow(%17, %18, %20)::Any
│   %22 = (%16 + %21)::Any
│   %23 = (%11 + %22)::Any
│   %24 = Core.get_binding_type(Ma